In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

**Summary**  

Decision trees can be used for both regression and classification problems. Think of decision trees as playing a [game of twenty questions](https://en.wikipedia.org/wiki/Twenty_Questions), where you have to arrive at an answer by asking no more than twenty questions and your next question depends on the answers to your previous questions. 

Mathematically, they split the predictor space into rectangular regions (refer Figure 8.3, page 308, PDF page 322), of possibly different sizes, such that the RSS in regression problems and the classification error rate in classification problems is minimized. That is, regression trees predict the mean response of the training observations that belong to the same region, and classification trees predict that each observation belongs to the most commonly occurring (mode) class of training observations in the region to which it belongs. (The variables used for splits are called *internal nodes*.)

* RSS is defined as $\sum \left(y_i - \hat{y}_{R_1} \right)+\sum \left(y_i - \hat{y}_{R_2} \right)+\cdots$, where $\hat{y}$ is the mean response of observations in each of the regions $R_1, R_2, \dots$.
* Classification error rate is the fraction of observations in a region that do not belong to the most common class. Gini index and entropy serve as sophisticated classification error rate, where they measure the total variance across the classes.

**Algorithm Biases** 

* Since decision trees make top-down decisions, they produce trees with good data splits at the top as opposed to bad splits at the top, even though both trees represent the same model.
* This results to the trees being shorter since the trees ask good questions at the beginning. Note that larger trees are indicative of over-fitting the data, since they result from asking too many questions about the data.
* Decision trees prefer correct models over incorrect models. That is, it will prefer a tree with not-so-good splits at the top but gives correct answers over a tree that has good splits at the top but produces incorrect answers.

In [2]:
data = pd.read_csv("../../data/csv/Carseats.csv")
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
data.isnull().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [4]:
data['ShelveLoc'].unique()

array(['Bad', 'Good', 'Medium'], dtype=object)

In [5]:
data['Urban'].unique()

array(['Yes', 'No'], dtype=object)

In [6]:
data['US'].unique()

array(['Yes', 'No'], dtype=object)

LabelEncoder() assigns numbers to alphabetically sorted data. Eg: [C, A, B] $\to$ [3, 1, 2].  This implies a natural ordering of the values and it is appropriate to order Bad, Good, Medium in an increasing manner.

In [7]:
data = data.apply(LabelEncoder().fit_transform)
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,255,49,51,11,141,54,0,17,7,1,1
1,297,22,27,16,129,18,1,40,0,1,1
2,267,24,14,10,138,15,2,34,2,1,1
3,158,28,77,4,249,31,2,30,4,1,1
4,37,52,42,3,178,62,0,13,3,1,0


Create a binary variable High (sales), where High$=1$ if Sales$>8$; High$=0$ if Sales$\le8$.

In [8]:
data['High'] = data['Sales'].map(lambda x: 1 if x>8 else 0)
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,High
0,255,49,51,11,141,54,0,17,7,1,1,1
1,297,22,27,16,129,18,1,40,0,1,1,1
2,267,24,14,10,138,15,2,34,2,1,1,1
3,158,28,77,4,249,31,2,30,4,1,1,1
4,37,52,42,3,178,62,0,13,3,1,0,1


In [9]:
data['High'].value_counts()

1    391
0      9
Name: High, dtype: int64

In [10]:
y = data['High']
x = data.loc[:, ~data.columns.isin(['Sales', 'High'])]
xTrainVal, xTestVal, yTrainVal, yTestVal = train_test_split(x, y, test_size=0.5)

In [11]:
model = DecisionTreeClassifier()
model.fit(xTrainVal, yTrainVal)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

The true positive rate (Recall) is 100% in the training data and 97% in the testing data.

In [12]:
# Training error
print(classification_report(yTrainVal, model.predict(xTrainVal)))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00       195

avg / total       1.00      1.00      1.00       200



In [13]:
model.score(xTrainVal, yTrainVal)

1.0

In [14]:
# Testing error
print(classification_report(yTestVal, model.predict(xTestVal)))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         4
          1       0.98      0.99      0.98       196

avg / total       0.96      0.97      0.97       200



In [15]:
model.score(xTestVal, yTestVal)

0.96999999999999997

In [16]:
# Feature importance
sorted(zip(map(lambda x: round(x, 2), model.feature_importances_), x.columns), reverse=True)

[(0.5, 'Price'),
 (0.32, 'Age'),
 (0.09, 'Population'),
 (0.09, 'CompPrice'),
 (0.0, 'Urban'),
 (0.0, 'US'),
 (0.0, 'ShelveLoc'),
 (0.0, 'Income'),
 (0.0, 'Education'),
 (0.0, 'Advertising')]

Notice that decision trees have high variance. That is, running the algorithm multiple times to randomly split training and validation data returns different predictions and feature importance score.

In [17]:
for k in range(1,5):
    xTrainVal, xTestVal, yTrainVal, yTestVal = train_test_split(x, y, test_size=0.5)
    model = DecisionTreeClassifier()
    model.fit(xTrainVal, yTrainVal)
    print "Training error: {}".format(model.score(xTrainVal, yTrainVal))
    print "Testing error: {}".format(model.score(xTestVal, yTestVal))
    print sorted(zip(map(lambda x: round(x, 2), model.feature_importances_), x.columns), reverse=True)
    print 

Training error: 1.0
Testing error: 0.97
[(0.42, 'Age'), (0.25, 'Price'), (0.23, 'Advertising'), (0.1, 'Population'), (0.0, 'Urban'), (0.0, 'US'), (0.0, 'ShelveLoc'), (0.0, 'Income'), (0.0, 'Education'), (0.0, 'CompPrice')]

Training error: 1.0
Testing error: 0.96
[(0.31, 'CompPrice'), (0.29, 'Income'), (0.16, 'Price'), (0.13, 'Advertising'), (0.11, 'Population'), (0.0, 'Urban'), (0.0, 'US'), (0.0, 'ShelveLoc'), (0.0, 'Education'), (0.0, 'Age')]

Training error: 1.0
Testing error: 0.98
[(0.49, 'Price'), (0.28, 'Age'), (0.14, 'ShelveLoc'), (0.09, 'Population'), (0.01, 'CompPrice'), (0.0, 'Urban'), (0.0, 'US'), (0.0, 'Income'), (0.0, 'Education'), (0.0, 'Advertising')]

Training error: 1.0
Testing error: 0.97
[(0.43, 'Age'), (0.36, 'Income'), (0.14, 'CompPrice'), (0.08, 'Price'), (0.0, 'Urban'), (0.0, 'US'), (0.0, 'ShelveLoc'), (0.0, 'Population'), (0.0, 'Education'), (0.0, 'Advertising')]

